In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters

In [3]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset


In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 22.3MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 621kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 5.60MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.91MB/s]


# Fully connected neural network with one hidden layer


In [5]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# Train the model


In [7]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.4090
Epoch [1/5], Step [200/600], Loss: 0.2312
Epoch [1/5], Step [300/600], Loss: 0.3114
Epoch [1/5], Step [400/600], Loss: 0.1953
Epoch [1/5], Step [500/600], Loss: 0.1704
Epoch [1/5], Step [600/600], Loss: 0.1068
Epoch [2/5], Step [100/600], Loss: 0.2144
Epoch [2/5], Step [200/600], Loss: 0.0814
Epoch [2/5], Step [300/600], Loss: 0.0932
Epoch [2/5], Step [400/600], Loss: 0.1152
Epoch [2/5], Step [500/600], Loss: 0.0765
Epoch [2/5], Step [600/600], Loss: 0.0541
Epoch [3/5], Step [100/600], Loss: 0.0798
Epoch [3/5], Step [200/600], Loss: 0.1121
Epoch [3/5], Step [300/600], Loss: 0.0899
Epoch [3/5], Step [400/600], Loss: 0.1772
Epoch [3/5], Step [500/600], Loss: 0.0672
Epoch [3/5], Step [600/600], Loss: 0.1104
Epoch [4/5], Step [100/600], Loss: 0.0489
Epoch [4/5], Step [200/600], Loss: 0.0361
Epoch [4/5], Step [300/600], Loss: 0.0275
Epoch [4/5], Step [400/600], Loss: 0.0253
Epoch [4/5], Step [500/600], Loss: 0.0839
Epoch [4/5], Step [600/600], Loss:

# Test the model

In [8]:
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 97.34 %
